## Other Odds and Ends

I hope the last few guest lectures have been interesting and informative!

Today, class will be in two parts. First, I'll introduce you to a few more modules, but ones that are not already installed, so you learn how to access modules other people have created. That way, you can search up whatever module you need and install it. We will install [**BioPython**](http://biopython.org/), a module with many functions for reading and parsing sequence data, and [**msprime**](https://msprime.readthedocs.io/en/stable/), which is a library that simulates genetic data using coalescent theory. 


### Biopython

One of the things we're teaching you in this course is how to use existing biology-centric modules in python so you don't have to re-invent the wheel! (Although, sometimes, re-inventing the wheel can be a useful teaching exercise...). Today, we'll be exploring the **Biopython** module. Biopython is a package with a collection of useful tools for bioinformatics. It can do things like parse sequence files in a variety of different formats (.fasta, .fastq, clustal alignment files, ExPASy files, Swiss Prot, etc), provide code to manipulate sequences (for example, perform transcription, translation, or reverse-complement), and interface with many useful databases on NCBI (Entrez, Pubmed, Blast).

To install Biopython, we are going to use the package manager pip. Pip's name is a recursive acronym that stands for "Pip Installs Packages".

In [ ]:
#%%bash
##I recommend doing this in Terminal!
#pip install biopython

Now, if you try to import the module (**Bio**), if no error is thrown, then the installation was successful!

In [ ]:
import Bio

## Using the SeqIO Module

One of the more useful sub-modules is SeqIO, which does input/output for sequences (and, I'm sorry to say, probably does it better than the FASTA parsers we've written thus far). SeqIO has a special sequence "class" that associates various attributes with sequence objects (some are methods, and some are variables). Let's look at an example parsing the cerevisiae_genome.fasta file from the first day of class.

In [ ]:
from Bio import SeqIO
 
# SeqIO.parse will parse a fasta file
# It returns an iterator object of sequence records
# each with its own attributes
pD="/Users/melyang/Desktop/PythonBootcamp2017/resources/"
fastaFile = pD+'cerevisiae_genome.fasta'
for record in SeqIO.parse(fastaFile,format="fasta"):
    print record
    print
    break

So you can see that each of these record objects holds quite a bit of information. Each record object also has special methods associated with it that you can use to pull specific bits of data about each sequence record. For example:

In [ ]:
pD="/Users/melyang/Desktop/PythonBootcamp2017/resources/"
fastaFile = pD+'cerevisiae_genome.fasta'
for record in SeqIO.parse(fastaFile,format="fasta"):
    print "This is the name of the sequence record: ",record.name
    # The .seq method references a sequence object
    # That is indexed and can therefore be sliced
    print "these are the first 50 bases:"
    print record.seq[0:50]
    print type(record.seq)
    print
    break

Though the data type for the sequences is not a string, this is not usually an issue, as most methods and functions you apply to strings also will work for the Seq data type. This is all I will say about this module now, but you are free to explore the **Bio** module yourself!

### msprime

The other module is **msprime**. 

Let's 

```bash
pip install msprime
```

in the Terminal. 

We are just going to try an example or two from their Tutorial!

In [ ]:
import msprime

tree_sequence = msprime.simulate(sample_size=5, Ne=1000)
print tree_sequence

The module **msprime** has the function **simulate**, which you use to simulate data. These data can be trees, as the above example shows, or sequence data, as we will show momentarily. 

Note that I put in arguments of a sample size of 5 and an effective population size of 1000. Then, what I am returned is a data object that is a 'TreeSequence' type. 

In [ ]:
print tree_sequence.trees()
for tree in tree_sequence.trees(): 
    tree.draw("tree_ex1.svg")
    print tree

from IPython.core.display import SVG
SVG("tree_ex1.svg")

The TreeSequence type has a method called **trees** that allows you to pull out the tree, represented here by a dictionary. In the dictionary, the key indicates the node, and the value is the parent node to which the key is attached. A -1 as the value indicates there is no parent node - you are at the root of the tree. 

The **tree** data type also has many methods, to get the parent node of another node, to retrieve an individual branch length, or to get the total branch length, for instance.

In [ ]:
print tree_sequence.trees()
for tree in tree_sequence.trees(): 
    tree.draw("tree_ex1.svg")
    print tree
    print 'parent of 4:', tree.get_parent(4)
    print 'children of 5:', tree.get_children(5)
    print 'total branch length:', tree.get_total_branch_length()
    print 'branch length for 5 (leading to parent):', tree.get_branch_length(2)
    print 'time in gen until 4, starting from present:',tree.get_time(4)

from IPython.core.display import SVG
SVG("tree_ex1.svg")

## Mutations 
These simulated trees are not that interesting until we get some variation in them! Mutations are what we use to study DNA, and thus, it's important to throw in a few in our simulations, to study the how the distribution of mutations change in different circumstances. 

The method **get_num_mutations** acts on the tree_sequence variable, to indicate how many total mutations were found. The **mutations** method acting on the tree variable returns a list of the Mutation data type, containing the position where the mutation occured, the node the mutation is found in, and the zero-based index of the mutation in the list. 

Without a length of sequence, however, you would have zero mutations! Thus, it is also important to mention the length of the sequence you are on. The mutation rate is in the unit "per base pair per generation". 

In [ ]:
import msprime

tree_sequence = msprime.simulate(sample_size=5, 
                                 Ne=1000, 
                                 length=1e4, 
                                 mutation_rate=2e-08)
print "Total mutations = ", tree_sequence.get_num_mutations()
for treeind,tree in enumerate(tree_sequence.trees()): 
    tree.draw("tree_ex2.svg")
    print "Tree",treeind,list(tree.mutations())

from IPython.core.display import SVG
SVG("tree_ex2.svg")

## Simulating sequences

We might want sequence data instead. Below is an example:

In [ ]:
import msprime
import numpy as np

def variants_example():
    tree_sequence = msprime.simulate(
        sample_size=20, Ne=1e4, length=5e3, recombination_rate=2e-8,
        mutation_rate=2e-8, random_seed=10)
    print("Simulated ", tree_sequence.get_num_mutations(), "mutations")
    for variant in tree_sequence.variants():
        print variant.index, variant.position, variant.genotypes
print variants_example()

Here, we add several more arguments to **tree_sequence**. For instance, we add in the recombination_rate argument, specifying the recombination rate per base per generation. We also specify a random_seed, which tells exactly where to start simulation from. Thus, this number will tell us how to get back the exact same simulation, if needed. 

Now, if we call on the method **variants**, there is a Variant data type that contains the index, position and array of genotypes associated with each mutation. In the example above, there are seven mutations at the specified positions along the sequence, and the array specifies how many individuals have the mutation (number of 1s in the array). 

Below, we show how to create an array containing all the genotypes for each mutation. 

In [ ]:
import msprime
import numpy as np

def variants_example():
    tree_sequence = msprime.simulate(
        sample_size=20, Ne=1e4, length=5e3, recombination_rate=2e-8,
        mutation_rate=2e-8, random_seed=10)
    print("Simulated ", tree_sequence.get_num_mutations(), "mutations")
    
    ##Get dimensions of array and initialize array.
    shape = tree_sequence.get_num_mutations(), tree_sequence.get_sample_size()
    mygenos=np.zeros(shape)
    
    for variant in tree_sequence.variants():
        #print variant.index, variant.position, variant.genotypes
        
        ##Add row of genotypes to correct row in array.
        mygenos[variant.index,]=variant.genotypes
    ##Return the array    
    return mygenos

print variants_example()

Now let's consider different population sizes!

In [ ]:
% matplotlib inline
import msprime
import numpy as np
import matplotlib.pyplot as plt

Ne=1e5
iterations=100

mycounts=np.zeros(iterations)
for i in range(iterations):
    tree_sequence = msprime.simulate(
            sample_size=20, Ne=Ne, length=5e3, recombination_rate=2e-8,
            mutation_rate=2e-8)
    mycounts[i] = tree_sequence.get_num_mutations()

fig=plt.figure()
ax=fig.add_subplot(1,1,1)
myhist = ax.hist(mycounts)
print myhist
ax.text(myhist[1].min(),myhist[0].max(),
        "mean = %.2f\nsd = %.2f" % (mycounts.mean(),mycounts.std()),
        verticalalignment='top',horizontalalignment='left')

plt.show()

We have done several things above! We calculate the number of segregating sites (or the total number of mutations) using **get_num_mutations**. Then, we run this simulation several times (100 in the example), so we simulate many replicates of the number of segregating sites. Then, we send these values to an array, **mycounts**, and plot a histogram for the number of segregating sites. We add text to the figure in the top left (how did I keep it in the top left for different window sizes?) specifying the mean and standard deviation. 

When we try different **Ne**, we find that we have more mutations for larger population sizes and much less mutations for smaller population sizes. Let's make one more plot.

In [ ]:
% matplotlib inline
import msprime
import numpy as np
import matplotlib.pyplot as plt

Ne=1e5
iterations=100
nes=[1e2,1e3,1e4,1e5,1e6]
mymeans = []
mysds = []
for neind,Ne in enumerate(nes):
    mycounts=np.zeros(iterations)
    for i in range(iterations):
        tree_sequence = msprime.simulate(
                sample_size=20, Ne=Ne, length=5e3, recombination_rate=2e-8,
                mutation_rate=2e-8)
        mycounts[i] = tree_sequence.get_num_mutations()
    mymeans.append(mycounts.mean())
    mysds.append(mycounts.std()) 

fig=plt.figure()
ax=fig.add_subplot(1,1,1)
ax.plot(nes,mymeans,'o')
#ax.set_xscale('log')
#ax.set_yscale('log')
plt.show()

Notice that we see an increase in mutations with an increase in population size. If we add a log x- and y- scale, we see a linear relationship between the x- and y- axis. This makes sense, since we are increasing each population size by an order of magnitude.

How can we modify the figure to make it more reader friendly?